<a href="https://colab.research.google.com/github/soohyoen/artificial-intelligence/blob/main/ex_0102b_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 나이브 베이즈 분류기 - 영화 리뷰.

**<span style="color:blue">목표</span>**: 베이즈 정리를 적용하여 자연어 분류 모형을 만들어 본다.

<br>
1. 베이즈 정리. <br>

 $P(A| W_1, W_2, W_3,...) = {P(W_1, W_2, W_3,...|A) \cdot P(A) \over P(W_1, W_2, W_3,....)}$ <br>
 $P(B| W_1, W_2, W_3,...) = {P(W_1, W_2, W_3,...|B) \cdot P(B) \over P(W_1, W_2, W_3,....)}$

<br>
2. 동일한 분모를 무시하면 다음 비례관계가 성립된다. <br>

 $P(A| W_1, W_2, W_3, \ldots) \sim P(W_1, W_2, W_3,...|A) \cdot P(A) $ <br>
 $P(B| W_1, W_2, W_3, \ldots) \sim P(W_1, W_2, W_3,...|B) \cdot P(B) $
 
<br>
3. 독립적인 단어 분포를 전제하면 다음과 같이 분해할 수 있다. <br>

 $P(A| W_1, W_2, W_3, \ldots) \sim  P(W_1|A)\cdot P(W_2|A)\cdot P(W_3|A) \cdots P(A) $ <br>
 $P(B| W_1, W_2, W_3, \ldots) \sim P(W_1|B)\cdot P(W_2|B)\cdot P(W_3|B) \cdots P(B) $

<br>
4. 이제는 로그함수를 적용해 본다. <br>

$Log_A \sim  Log(P(W_1|A)) +  Log(P(W_2|A)) + Log(P(W_3|A)) + \ldots + Log(P(A)) $ <br>
$Log_B \sim  Log(P(W_1|B)) +  Log(P(W_2|B)) + Log(P(W_3|B)) + \ldots + Log(P(B)) $
<br>

**<span style="color:blue">결론</span>**:  $Log_A$와 $Log_B$를 비교해서 큰 쪽으로 인식!

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.datasets import load_files

#### 1. 학습 자료를 읽어와서 전처리한다:

In [ ]:
# 구글 드라이브 마운트.
from google.colab import drive
drive.mount('/content/drive')  # 절차를 따라서 한다.

In [ ]:
# 경로 이동.
%cd "/content/drive/MyDrive/GwangjuAI/modelling/notebook"

In [ ]:
# 데이터 폴더 설정 후 읽어온다.
reviews = load_files('../data/txt_sentoken/')
my_docs, y = reviews.data, reviews.target

In [ ]:
# 0 유형의 리뷰를 가져온다. 
my_docs_0 = list(pd.Series(my_docs)[pd.Series(y) == 0])

# 1 유형의 리뷰를 가져온다. 
my_docs_1 = list(pd.Series(my_docs)[pd.Series(y) == 1])

In [ ]:
print(f"0 유형의 수 : {len(my_docs_0)}")
print(f"1 유형의 수 : {len(my_docs_1)}")

In [ ]:
#출력해 본다.
# Label 0 = negative 이며 Label 1 = positive 임을 확인할 수 있다.
#print(my_docs_0[0])
#print(my_docs_1[0])

In [ ]:
# 학습용과 테스트 용으로 분리해 놓는다.
n_train = 700
n_test = 1000 - n_train
my_docs_train_0 = my_docs_0[:n_train]
my_docs_train_1 = my_docs_1[:n_train]
my_docs_test = my_docs_0[n_train:] + my_docs_1[n_train:]
Y_test = [0]*n_test + [1]*n_test

In [ ]:
# 전처리 해서 단어별 도수표를 만들어주는 함수.
def preprocessor(a_doc):
    freq_dict = {}
    for a_line in a_doc:
        a_line = str(a_line)                  # 단순한 문자열로 변환.
        a_line = a_line.lower()               # 소문자화.
        a_line = re.sub(r"\W"," ",a_line)     # 특수문자 제거.
        a_line = re.sub(r"\d", " ", a_line)   # 숫자 제거.
        a_line = re.sub("a|the|and|or|because|at", " ",a_line)  #  불용어 제거.
        a_line = a_line.split()               # 분절.
        for a_word in a_line:
            if len(a_word) > 3:               # 길이가 최소 조건을 충족하는 단어만 사용.            
                if a_word in freq_dict:
                    freq_dict[a_word] += 1    # 카운트 누적.
                else:
                    freq_dict[a_word] = 2     # 처음 발견. 기본값 1 + 누적 1.
    return freq_dict

#### 2. 학습 모형을 준비해 둔다:

In [ ]:
# Series로 변환.
freq_0 = pd.Series(preprocessor(my_docs_train_0)).sort_values(ascending=False)
freq_1 = pd.Series(preprocessor(my_docs_train_1)).sort_values(ascending=False) 

In [ ]:
# Vocabulary 사전 크기 확인.
print("Size of 0: {}".format( len(freq_0)))
print("Size of 1: {}".format( len(freq_1)))

In [ ]:
# Vocabulary 수 맞춤.
n_voca = 15000
freq_0 = freq_0.iloc[:n_voca]
freq_1 = freq_1.iloc[:n_voca]

In [ ]:
# 로그 확률로 변환.
freq_0_sum = freq_0.sum()
freq_1_sum = freq_1.sum()
log_prob_0 = dict(np.log(freq_0/freq_0_sum))    # Log(P(W|0))
log_prob_1 = dict(np.log(freq_1/freq_1_sum))    # Log(P(W|1))

#### 3. 예측을 실시한다:

In [ ]:
Y_pred = []
for a_sentence in my_docs_test:
    log_prob_sum_0 = 0.
    log_prob_sum_1 = 0.
    a_sent_preprocessed = preprocessor([a_sentence])
    for a_word, a_freq in a_sent_preprocessed.items():
        if a_word in log_prob_0:
            log_prob_sum_0 += log_prob_0[a_word]*a_freq
        else:
            log_prob_sum_0 += np.log(1.0/freq_0_sum)*a_freq
            
        if a_word in log_prob_1:
            log_prob_sum_1 += log_prob_1[a_word]*a_freq
        else:
            log_prob_sum_1 += np.log(1.0/freq_1_sum)*a_freq
            
    if (log_prob_sum_0 > log_prob_sum_1):
        Y_pred.append(0)
    else:
        Y_pred.append(1)

In [ ]:
# 정확도 계산.
correct = pd.Series([ x == y for (x,y) in zip(Y_pred, Y_test) ])
print("Accuracy : {:.3f}".format(correct.mean()))